In [186]:
%cd /content/drive/MyDrive/Colab\ Notebooks/bigcontest/

/content/drive/MyDrive/Colab Notebooks/bigcontest


In [187]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import re
from typing import collections, Dict, List, Tuple
from statsmodels.tsa.arima_model import ARIMA
import statsmodels.api as sm
import warnings
warnings.filterwarnings(action='ignore') 

# train / validation data + weight data

## 데이터 로드

In [202]:
train = pd.read_excel('./train.xlsx')
validation = pd.read_excel('./validation.xlsx')

In [220]:
# 중량데이터 로드 
sq_qty = pd.read_csv('./weight_data/squid_qty.csv')
sal_qty = pd.read_csv('./weight_data/salmon_qty.csv')
shr_qty = pd.read_csv('./weight_data/shrimp_qty.csv')

sq_val_qty = pd.read_csv('./weight_data/squid_val_qty.csv')
sal_val_qty = pd.read_csv('./weight_data/salmon_val_qty.csv')
shr_val_qty = pd.read_csv('./weight_data/shrimp_val_qty.csv')

del sq_qty['Unnamed: 0']
del sal_qty['Unnamed: 0']
del shr_qty['Unnamed: 0']

del sq_val_qty['Unnamed: 0']
del sal_val_qty['Unnamed: 0']
del shr_val_qty['Unnamed: 0']

## 날짜변수 전처리

In [221]:
# 월 / 년도별 변수 생성 
train['date_m'] = train['REG_DATE'].apply(lambda x : str(x.to_period(freq='M')))
train['date_y'] = train['REG_DATE'].apply(lambda x : str(x.to_period(freq='A')))

validation['date_m'] = validation['REG_DATE'].apply(lambda x : str(x.to_period(freq='M')))
validation['date_y'] = validation['REG_DATE'].apply(lambda x : str(x.to_period(freq='A')))

In [222]:
# str타입의 날짜 컬럼 생성
train['date'] = train['REG_DATE'].apply(lambda x : x.strftime('%Y-%m-%d'))
validation['date'] = validation['REG_DATE'].apply(lambda x : x.strftime('%Y-%m-%d'))

## 상세어종에 맞게 train데이터 분류

In [223]:
sq = train[train['P_NAME'] == '오징어']
sal = train[train['P_NAME'] == '연어']
shr = train[train['P_NAME'] == '흰다리새우']

print('오징어 데이터: ',sq.shape)
print('연어 데이터: ',sal.shape)
print('새우 데이터: ',shr.shape)

오징어 데이터:  (2150, 13)
연어 데이터:  (1489, 13)
새우 데이터:  (2601, 13)


In [224]:
sq_val = validation[validation['P_NAME'] == '오징어']
sal_val = validation[validation['P_NAME'] == '연어']
shr_val = validation[validation['P_NAME'] == '흰다리새우']

print('오징어 데이터: ',sq_val.shape)
print('연어 데이터: ',sal_val.shape)
print('새우 데이터: ',shr_val.shape)

오징어 데이터:  (621, 13)
연어 데이터:  (406, 13)
새우 데이터:  (532, 13)


## 중량데이터 전처리 

- train 데이터는 수치형 컬럼을 재외한채로 중복된 데이터가 존재하지 않음 
- 하지만 외부 데이터(중량)은 수치형 컬럼을 제회하더라도 중복된 데이터가 존재함. 

- 이는 train 데이터와 외부데이터를 merge할떄 적은쪽의 데이터가 중복된 데이터의 첫번째 컬럼과 merge되는 현상이 발생 

- 따라서 외부데이터에서 'date', 'date_m', 'date_y', 'CTRY_1', 'CTRY_2', 'P_NAME', 'P_IMPORT_TYPE' 이 같다면 
  하나로 합치는 작업이 필요함

### P_NAME컬럼 생성

In [225]:
# P_NAME column 생성
sq_qty['P_NAME'] = sq_qty['대표품목별'].apply(lambda x : re.sub(r'[^가-힣,]', ' ', x).split()[0])
sal_qty['P_NAME'] = sal_qty['대표품목별'].apply(lambda x : re.sub(r'[^가-힣,]', ' ', x).split()[0])
shr_qty['P_NAME'] = shr_qty['대표품목별'].apply(lambda x : re.sub(r'[^가-힣,]', ' ', x).split()[0])

# validation data에도 적용
sq_val_qty['P_NAME'] = sq_val_qty['대표품목별'].apply(lambda x : re.sub(r'[^가-힣,]', ' ', x).split()[0])
sal_val_qty['P_NAME'] = sal_val_qty['대표품목별'].apply(lambda x : re.sub(r'[^가-힣,]', ' ', x).split()[0])
shr_val_qty['P_NAME'] = shr_val_qty['대표품목별'].apply(lambda x : re.sub(r'[^가-힣,]', ' ', x).split()[0])

### P_IMPORT_TYPE 컬럼 생성

In [226]:
# P_IMPORT_TYPE 생성 
sq_qty['P_IMPORT_TYPE'] = sq_qty['대표품목별'].apply(lambda x : re.sub(r'[^가-힣,]', ' ', x).split()[1])
sal_qty['P_IMPORT_TYPE'] = sal_qty['대표품목별'].apply(lambda x : re.sub(r'[^가-힣,]', ' ', x).split()[1])
shr_qty['P_IMPORT_TYPE'] = shr_qty['대표품목별'].apply(lambda x : re.sub(r'[^가-힣,]', ' ', x).split()[1])

# validation data에도 생성
sq_val_qty['P_IMPORT_TYPE'] = sq_val_qty['대표품목별'].apply(lambda x : re.sub(r'[^가-힣,]', ' ', x).split()[1])
sal_val_qty['P_IMPORT_TYPE'] = sal_val_qty['대표품목별'].apply(lambda x : re.sub(r'[^가-힣,]', ' ', x).split()[1])
shr_val_qty['P_IMPORT_TYPE'] = shr_val_qty['대표품목별'].apply(lambda x : re.sub(r'[^가-힣,]', ' ', x).split()[1])

### 변수명 변경

In [227]:
# 기존 train 데이터와 합치기 위해서 컬럼이름 변경
sq_qty.rename(columns={'Date':'date', '제조국(원산지)별':'CTRY_1','수출국별':'CTRY_2' ,'총계_중량(KG)':'WEIGHT(KG)', \
                      '총계_금액($)':'TOTAL_PRICE'}, inplace = True)

sq_val_qty.rename(columns={'Date':'date', '제조국(원산지)별':'CTRY_1','수출국별':'CTRY_2' ,'총계_중량(KG)':'WEIGHT(KG)', \
                      '총계_금액($)':'TOTAL_PRICE'}, inplace = True)

sal_qty.rename(columns={'Date':'date', '제조국(원산지)별':'CTRY_1','수출국별':'CTRY_2' ,'총계_중량(KG)':'WEIGHT(KG)', \
                      '총계_금액($)':'TOTAL_PRICE'}, inplace = True)

sal_val_qty.rename(columns={'Date':'date', '제조국(원산지)별':'CTRY_1','수출국별':'CTRY_2' ,'총계_중량(KG)':'WEIGHT(KG)', \
                      '총계_금액($)':'TOTAL_PRICE'}, inplace = True)

shr_qty.rename(columns={'Date':'date', '제조국(원산지)별':'CTRY_1','수출국별':'CTRY_2' ,'총계_중량(KG)':'WEIGHT(KG)', \
                      '총계_금액($)':'TOTAL_PRICE'}, inplace = True)
shr_val_qty.rename(columns={'Date':'date', '제조국(원산지)별':'CTRY_1','수출국별':'CTRY_2' ,'총계_중량(KG)':'WEIGHT(KG)', \
                      '총계_금액($)':'TOTAL_PRICE'}, inplace = True)

In [228]:
sq_qty

,date,제품구분별,CTRY_1,CTRY_2,수입용도별,대표품목별,WEIGHT(KG),TOTAL_PRICE,적합_중량(kg),적합_금액($),부적합_중량(kg),부적합_금액($),P_NAME,P_IMPORT_TYPE
0,2015-12-28,수산물,중국,중국,판매용,"오징어(냉동,동체,자숙)","15,500.000",-,"15,500.000",-,.000,.000,오징어,"냉동,동체,자숙"
1,2015-12-28,수산물,페루,페루,판매용,"오징어(냉동,다리)","22,400.000",-,"22,400.000",-,.000,.000,오징어,"냉동,다리"
2,2015-12-28,수산물,페루,페루,자사제품제조용,"오징어(냉동,다리)","23,800.000",-,"23,800.000",-,.000,.000,오징어,"냉동,다리"
3,2015-12-28,수산물,페루,페루,자사제품제조용,"오징어(냉동,동체,자숙)","22,560.000",-,"22,560.000",-,.000,.000,오징어,"냉동,동체,자숙"
4,2015-12-28,수산물,칠레,칠레,판매용,"오징어(냉동,동체)","48,000.000","48,000.000","48,000.000","48,000.000",.000,.000,오징어,"냉동,동체"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8071,2019-12-30,수산물,대만,대만,판매용,오징어(냉동),"24,309.000",-,"24,309.000",-,.000,.000,오징어,냉동
8072,2019-12-30,수산물,페루,페루,판매용,"오징어(냉동,다리)","9,240.000","21,976.000","9,240.000","21,976.000",.000,.000,오징어,"냉동,다리"
8073,2019-12-30,수산물,중국,중국,판매용,"오징어(냉동,동체)","37,000.000","147,700.000","37,000.000","147,700.000",.000,.000,오징어,"냉동,동체"
8074,2019-12-30,수산물,페루,페루,판매용,"오징어(냉동,동체)","62,350.000","140,950.000","62,350.000","140,950.000",.000,.000,오징어,"냉동,동체"


In [229]:
# str타입의 수치형 데이터 전처리

sq_qty['WEIGHT(KG)'] = sq_qty['WEIGHT(KG)'].apply(lambda x: float(x.replace(',','')))
sq_qty['TOTAL_PRICE'] = sq_qty['TOTAL_PRICE'].apply(lambda x: x.replace('-', '0'))
sq_qty['TOTAL_PRICE'] = sq_qty['TOTAL_PRICE'].apply(lambda x: x.replace(',', ''))
sq_qty['TOTAL_PRICE'] = sq_qty['TOTAL_PRICE'].apply(lambda x: float(x))

sal_qty['WEIGHT(KG)'] = sal_qty['WEIGHT(KG)'].apply(lambda x: float(x.replace(',','')))
sal_qty['TOTAL_PRICE'] = sal_qty['TOTAL_PRICE'].apply(lambda x: x.replace('-', '0'))
sal_qty['TOTAL_PRICE'] = sal_qty['TOTAL_PRICE'].apply(lambda x: x.replace(',', ''))
sal_qty['TOTAL_PRICE'] = sal_qty['TOTAL_PRICE'].apply(lambda x: float(x))

shr_qty['WEIGHT(KG)'] = shr_qty['WEIGHT(KG)'].apply(lambda x: float(x.replace(',','')))
shr_qty['TOTAL_PRICE'] = shr_qty['TOTAL_PRICE'].apply(lambda x: x.replace('-', '0'))
shr_qty['TOTAL_PRICE'] = shr_qty['TOTAL_PRICE'].apply(lambda x: x.replace(',', ''))
shr_qty['TOTAL_PRICE'] = shr_qty['TOTAL_PRICE'].apply(lambda x: float(x))

# validation data에도 적용
sq_val_qty['WEIGHT(KG)'] = sq_val_qty['WEIGHT(KG)'].apply(lambda x: float(x.replace(',','')))
sq_val_qty['TOTAL_PRICE'] = sq_val_qty['TOTAL_PRICE'].apply(lambda x: x.replace('-', '0'))
sq_val_qty['TOTAL_PRICE'] = sq_val_qty['TOTAL_PRICE'].apply(lambda x: x.replace(',', ''))
sq_val_qty['TOTAL_PRICE'] = sq_val_qty['TOTAL_PRICE'].apply(lambda x: float(x))

sal_val_qty['WEIGHT(KG)'] = sal_val_qty['WEIGHT(KG)'].apply(lambda x: float(x.replace(',','')))
sal_val_qty['TOTAL_PRICE'] = sal_val_qty['TOTAL_PRICE'].apply(lambda x: x.replace('-', '0'))
sal_val_qty['TOTAL_PRICE'] = sal_val_qty['TOTAL_PRICE'].apply(lambda x: x.replace(',', ''))
sal_val_qty['TOTAL_PRICE'] = sal_val_qty['TOTAL_PRICE'].apply(lambda x: float(x))

shr_val_qty['WEIGHT(KG)'] = shr_val_qty['WEIGHT(KG)'].apply(lambda x: float(x.replace(',','')))
shr_val_qty['TOTAL_PRICE'] = shr_val_qty['TOTAL_PRICE'].apply(lambda x: x.replace('-', '0'))
shr_val_qty['TOTAL_PRICE'] = shr_val_qty['TOTAL_PRICE'].apply(lambda x: x.replace(',', ''))
shr_val_qty['TOTAL_PRICE'] = shr_val_qty['TOTAL_PRICE'].apply(lambda x: float(x))

In [230]:
# groupby로 같은날의 같은 수입형태의 데이터는 하나로 합침
sq_qty = sq_qty.groupby(['date', 'CTRY_1', 'CTRY_2', 'P_NAME', 'P_IMPORT_TYPE'])\
                    [['WEIGHT(KG)', 'TOTAL_PRICE']].sum()
sq_qty.reset_index(inplace=True)


sal_qty = sal_qty.groupby(['date', 'CTRY_1', 'CTRY_2', 'P_NAME', 'P_IMPORT_TYPE'])\
                    [['WEIGHT(KG)', 'TOTAL_PRICE']].sum()
sal_qty.reset_index(inplace=True)


shr_qty = shr_qty.groupby(['date', 'CTRY_1', 'CTRY_2', 'P_NAME', 'P_IMPORT_TYPE'])\
                    [['WEIGHT(KG)', 'TOTAL_PRICE']].sum()
shr_qty.reset_index(inplace=True)


# validation data에도 적용
sq_val_qty = sq_val_qty.groupby(['date', 'CTRY_1', 'CTRY_2', 'P_NAME', 'P_IMPORT_TYPE'])\
                    [['WEIGHT(KG)', 'TOTAL_PRICE']].sum()
sq_val_qty.reset_index(inplace=True)


sal_val_qty = sal_val_qty.groupby(['date', 'CTRY_1', 'CTRY_2', 'P_NAME', 'P_IMPORT_TYPE'])\
                    [['WEIGHT(KG)', 'TOTAL_PRICE']].sum()
sal_val_qty.reset_index(inplace=True)


shr_val_qty = shr_val_qty.groupby(['date', 'CTRY_1', 'CTRY_2', 'P_NAME', 'P_IMPORT_TYPE'])\
                    [['WEIGHT(KG)', 'TOTAL_PRICE']].sum()
shr_val_qty.reset_index(inplace=True)

## 데이터 병합

In [231]:
# 기존 train데이터와 중량데이터 merge
df_sq = pd.merge(sq, sq_qty, how='inner', on=['date','CTRY_1', 'CTRY_2','P_IMPORT_TYPE','P_NAME'])
df_sal = pd.merge(sal, sal_qty, how='inner', on=['date','CTRY_1', 'CTRY_2','P_IMPORT_TYPE','P_NAME'])
df_shr = pd.merge(shr, shr_qty, how='inner', on=['date','CTRY_1', 'CTRY_2','P_IMPORT_TYPE','P_NAME'])

# 기존 validation데이터와 중량데이터 merge
df_sq_val = pd.merge(sq_val, sq_val_qty, how='inner', on=['date','CTRY_1', 'CTRY_2','P_IMPORT_TYPE','P_NAME'])
df_sal_val = pd.merge(sal_val, sal_val_qty, how='inner', on=['date','CTRY_1', 'CTRY_2','P_IMPORT_TYPE','P_NAME'])
df_shr_val = pd.merge(shr_val, shr_val_qty, how='inner', on=['date','CTRY_1', 'CTRY_2','P_IMPORT_TYPE','P_NAME'])


# train / validation data + temperature data

In [232]:
## 날짜별 온도를 dictionary 형태로 변환
def change(contury):
    tmp = []
    for t in contury['time']:
        tmp.append(t[:10])
    contury['date'] = tmp
    contury = contury.drop(['time'],axis=1)
    
    dict_ = {k:v for k, v in zip(tmp, list(contury["thetao"]))}
    
    return dict_

## 오징어 온도 데이터 병합(2015~2019)

In [233]:
path = './squid_temp/2015_2019/'
file_list = os.listdir(path)
print ("file_list: {}".format(file_list))

file_list: ['korea-china_temp.csv', 'peru_temp.csv', 'chile_temp.csv', 'argentina_temp.csv', 'equador_temp.csv', 'newzeal_temp.csv', 'taiwan_temp.csv', 'banuatoo_temp.csv', 'russia_temp.csv']


In [234]:
korea_china = pd.read_csv(path+file_list[0], skiprows=[0,1,2,3,4,5,6])
peru = pd.read_csv(path+file_list[1], skiprows=[0,1,2,3,4,5,6])
chile = pd.read_csv(path+file_list[2], skiprows=[0,1,2,3,4,5,6])
argentina = pd.read_csv(path+file_list[3], skiprows=[0,1,2,3,4,5,6])
equador = pd.read_csv(path+file_list[4], skiprows=[0,1,2,3,4,5,6])
newzeal = pd.read_csv(path+file_list[5], skiprows=[0,1,2,3,4,5,6])
taiwan = pd.read_csv(path+file_list[6], skiprows=[0,1,2,3,4,5,6])
banuatoo = pd.read_csv(path+file_list[7], skiprows=[0,1,2,3,4,5,6])
russia = pd.read_csv(path+file_list[8], skiprows=[0,1,2,3,4,5,6])

In [235]:
korea_china = change(korea_china)
peru = change(peru)
chile = change(chile)
argentina = change(argentina)
equador = change(equador)
newzeal = change(newzeal)
taiwan = change(taiwan)
banuatoo = change(banuatoo)
russia = change(russia)

In [236]:
df_sq['CTRY_1'].unique()

array(['중국', '페루', '칠레', '대만', '대한민국', '뉴질랜드', '아르헨티나', '바누아투', '에콰도르',
       '러시아'], dtype=object)

In [237]:
temp = []
for day, crt in zip(df_sq['date'], df_sq['CTRY_1']):
  if crt == "중국":
    temp.append(korea_china[day])
  elif crt == "페루":
    temp.append(peru[day])
  elif crt == "칠레":
    temp.append(chile[day])
  elif crt == "대만":
    temp.append(taiwan[day])
  elif crt == "대한민국":
    temp.append(korea_china[day])
  elif crt == "뉴질랜드":
    temp.append(newzeal[day])
  elif crt == "아르헨티나":
    temp.append(argentina[day])
  elif crt == "바누아투":
    temp.append(banuatoo[day])
  elif crt == "에콰도르":
    temp.append(equador[day])
  else:
    temp.append(russia[day])
        

In [238]:
df_sq['temp']=temp

In [239]:
df_sq

,REG_DATE,P_TYPE,CTRY_1,CTRY_2,P_PURPOSE,CATEGORY_1,CATEGORY_2,P_NAME,P_IMPORT_TYPE,P_PRICE,date_m,date_y,date,WEIGHT(KG),TOTAL_PRICE,temp
0,2015-12-28,수산물,중국,중국,판매용,연체류 해물모듬,오징어,오징어,"냉동,동체",0.793729,2015-12,2015,2015-12-28,33475.0,29328.00,4.435041
1,2015-12-28,수산물,페루,페루,판매용,연체류 해물모듬,오징어,오징어,"냉동,다리",1.233434,2015-12,2015,2015-12-28,46200.0,0.00,14.602099
2,2015-12-28,수산물,페루,페루,판매용,연체류 해물모듬,오징어,오징어,"냉동,동체,자숙",5.481140,2015-12,2015,2015-12-28,66560.0,189200.00,14.602099
3,2015-12-28,수산물,칠레,칠레,판매용,연체류 해물모듬,오징어,오징어,"냉동,지느러미",0.762758,2015-12,2015,2015-12-28,48000.0,40080.00,14.845271
4,2015-12-28,수산물,칠레,칠레,판매용,연체류 해물모듬,오징어,오징어,"냉동,동체",0.962386,2015-12,2015,2015-12-28,48000.0,48000.00,14.845271
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1328,2019-12-30,수산물,페루,페루,판매용,연체류 해물모듬,오징어,오징어,"냉동,다리",2.425554,2019-12,2019,2019-12-30,9240.0,21976.00,14.967589
1329,2019-12-30,수산물,중국,중국,판매용,연체류 해물모듬,오징어,오징어,"냉동,동체",3.906298,2019-12,2019,2019-12-30,37000.0,147700.00,8.451765
1330,2019-12-30,수산물,페루,페루,판매용,연체류 해물모듬,오징어,오징어,"냉동,지느러미",1.760060,2019-12,2019,2019-12-30,4410.0,0.00,14.967589
1331,2019-12-30,수산물,페루,페루,판매용,연체류 해물모듬,오징어,오징어,"냉동,동체",2.281843,2019-12,2019,2019-12-30,62350.0,140950.00,14.967589


## 오징어 온도 데이터 병합(2020)

In [240]:
path = './squid_temp/2020/'
file_list = os.listdir(path)
print ("file_list: {}".format(file_list))

file_list: ['2020taiwan_temp.csv', '2020newzeal_temp.csv', '2020equador_temp.csv', '2020argentina_temp.csv', '2020chile_temp.csv', '2020peru_temp.csv', '2020korea-china_temp.csv', '2020uruguy_temp.csv', '2020indonesia_temp.csv', '2020russia_temp.csv', '2020cananda_temp.csv', '2020usa_temp.csv']


In [241]:
taiwan = pd.read_csv(path+file_list[0], skiprows=[0,1,2,3,4,5,6])
newzeal = pd.read_csv(path+file_list[1], skiprows=[0,1,2,3,4,5,6])
equador = pd.read_csv(path+file_list[2], skiprows=[0,1,2,3,4,5,6])
argentina = pd.read_csv(path+file_list[3], skiprows=[0,1,2,3,4,5,6])
chile = pd.read_csv(path+file_list[4], skiprows=[0,1,2,3,4,5,6])
peru = pd.read_csv(path+file_list[5], skiprows=[0,1,2,3,4,5,6])
korea_china = pd.read_csv(path+file_list[6], skiprows=[0,1,2,3,4,5,6])
uruguy = pd.read_csv(path+file_list[7], skiprows=[0,1,2,3,4,5,6])
indonesia = pd.read_csv(path+file_list[8], skiprows=[0,1,2,3,4,5,6])
russia = pd.read_csv(path+file_list[9], skiprows=[0,1,2,3,4,5,6])
cananda = pd.read_csv(path+file_list[10], skiprows=[0,1,2,3,4,5,6])
usa = pd.read_csv(path+file_list[11], skiprows=[0,1,2,3,4,5,6])

In [242]:
taiwan = change(taiwan)
newzeal = change(newzeal)
equador = change(equador)
argentina = change(argentina)
chile = change(chile)
peru = change(peru)
korea_china = change(korea_china)
uruguy = change(uruguy)
indonesia = change(indonesia)
russia = change(russia)
cananda = change(cananda)
usa = change(usa)

In [243]:
df_sq_val['CTRY_1'].unique()

array(['페루', '칠레', '중국', '대만', '러시아', '인도네시아', '미국', '아르헨티나', '대한민국',
       '뉴질랜드', '캐나다', '우루과이'], dtype=object)

In [244]:
temp = []
for day, crt in zip(df_sq_val['date'], df_sq_val['CTRY_1']):
  if crt == "페루":
    temp.append(peru[day])
  elif crt == "칠레":
    temp.append(chile[day])
  elif crt == "중국":
    temp.append(korea_china[day])
  elif crt == "대만":
    temp.append(taiwan[day])
  elif crt == "러시아":
    temp.append(russia[day])
  elif crt == "인도네시아":
    temp.append(indonesia[day])
  elif crt == "미국":
    temp.append(usa[day])
  elif crt == "아르헨티나":
    temp.append(argentina[day])
  elif crt == "대한민국":
    temp.append(korea_china[day])
  elif crt == "뉴질랜드":
    temp.append(newzeal[day])
  elif crt == "캐나다":
    temp.append(cananda[day])
  else:
    temp.append(uruguy[day])

In [245]:
df_sq_val['temp']=temp

## 연어 온도 데이터 병합(2015_2019)

In [246]:
path = './salmon_temp/2015_2019/'
file_list = os.listdir(path)
print ("file_list: {}".format(file_list))

file_list: ['uk_temp.csv', 'norway_temp.csv', 'usa_temp.csv', 'aus_temp.csv', 'canada_temp.csv', 'chile_temp.csv', 'ireland_temp.csv', 'island_temp.csv', 'russia_temp.csv']


In [247]:
uk = pd.read_csv(path+file_list[0], skiprows=[0,1,2,3,4,5,6])
norway = pd.read_csv(path+file_list[1], skiprows=[0,1,2,3,4,5,6])
usa = pd.read_csv(path+file_list[2], skiprows=[0,1,2,3,4,5,6])
aus = pd.read_csv(path+file_list[3], skiprows=[0,1,2,3,4,5,6])
canada = pd.read_csv(path+file_list[4], skiprows=[0,1,2,3,4,5,6])
chile = pd.read_csv(path+file_list[5], skiprows=[0,1,2,3,4,5,6])
ireland = pd.read_csv(path+file_list[6], skiprows=[0,1,2,3,4,5,6])
island = pd.read_csv(path+file_list[7], skiprows=[0,1,2,3,4,5,6])
russia = pd.read_csv(path+file_list[8], skiprows=[0,1,2,3,4,5,6])

In [248]:
uk = change(uk)
norway = change(norway)
usa = change(usa)
aus = change(aus)
canada = change(canada)
chile = change(chile)
ireland = change(ireland)
island = change(island)
russia = change(russia)

In [249]:
df_sal['CTRY_1'].unique()

array(['노르웨이', '캐나다', '영국', '칠레', '아일랜드', '미국', '러시아', '호주', '아이슬란드'],
      dtype=object)

In [250]:
temp = []
for day, crt in zip(df_sal['date'], df_sal['CTRY_1']):
  if crt == "노르웨이":
    temp.append(norway[day])
  elif crt == "캐나다":
    temp.append(canada[day])
  elif crt == "영국":
    temp.append(uk[day])
  elif crt == "칠레":
    temp.append(chile[day])
  elif crt == "아일랜드":
    temp.append(ireland[day])
  elif crt == "미국":
    temp.append(usa[day])
  elif crt == "러시아":
    temp.append(russia[day])
  elif crt == "호주":
    temp.append(aus[day])
  else:
    temp.append(island[day])

In [251]:
df_sal['temp']=temp

## 연어 온도 데이터 병합(2020)

In [252]:
path = './salmon_temp/2020/'
file_list = os.listdir(path)
print ("file_list: {}".format(file_list))

file_list: ['2020canada_temp.csv', '2020aus_temp.csv', '2020russia_temp.csv', '2020island_temp.csv', '2020chile_temp.csv', '2020usa_temp.csv', '2020norway_temp.csv', '2020uk_temp.csv']


In [253]:
canada = pd.read_csv(path+file_list[0], skiprows=[0,1,2,3,4,5,6])
aus = pd.read_csv(path+file_list[1], skiprows=[0,1,2,3,4,5,6])
russia = pd.read_csv(path+file_list[2], skiprows=[0,1,2,3,4,5,6])
island = pd.read_csv(path+file_list[3], skiprows=[0,1,2,3,4,5,6])
chile = pd.read_csv(path+file_list[4], skiprows=[0,1,2,3,4,5,6])
usa = pd.read_csv(path+file_list[5], skiprows=[0,1,2,3,4,5,6])
norway = pd.read_csv(path+file_list[6], skiprows=[0,1,2,3,4,5,6])
uk = pd.read_csv(path+file_list[7], skiprows=[0,1,2,3,4,5,6])

In [254]:
canada = change(canada)
aus = change(aus)
russia = change(russia)
island = change(island)
chile = change(chile)
usa = change(usa)
norway = change(norway)
uk = change(uk)

In [255]:
df_sal_val['CTRY_1'].unique()

array(['노르웨이', '영국', '아이슬란드', '캐나다', '칠레', '호주', '미국'], dtype=object)

In [256]:
temp = []
for day, crt in zip(df_sal_val['date'], df_sal_val['CTRY_1']):
  if crt == "노르웨이":
    temp.append(norway[day])
  elif crt == "영국":
    temp.append(uk[day])
  elif crt == "아이슬란드":
    temp.append(island[day])
  elif crt == "캐나다":
    temp.append(canada[day])
  elif crt == "칠레":
    temp.append(chile[day])
  elif crt == "호주":
    temp.append(aus[day])
  else:
    temp.append(usa[day])


In [257]:
df_sal_val['temp']=temp

## 흰다리새우 온도 데이터 병합(2015~2019)

In [258]:
path = './shrimp_temp/2015_2019/'
file_list = os.listdir(path)
print ("file_list: {}".format(file_list))

file_list: ['china_temp.csv', 'saudi_temp.csv', 'india_temp.csv', 'indonesia_temp.csv', 'equador_temp.csv', 'philipines_temp.csv', 'thai_temp.csv', 'vietnam_temp.csv', 'malay_temp.csv', 'panama_temp.csv', 'peru_temp.csv', 'columbia_temp.csv']


In [259]:
china = pd.read_csv(path+file_list[0], skiprows=[0,1,2,3,4,5,6])
saudi = pd.read_csv(path+file_list[1], skiprows=[0,1,2,3,4,5,6])
india = pd.read_csv(path+file_list[2], skiprows=[0,1,2,3,4,5,6])
indonesia = pd.read_csv(path+file_list[3], skiprows=[0,1,2,3,4,5,6])
equador = pd.read_csv(path+file_list[4], skiprows=[0,1,2,3,4,5,6])
philipines = pd.read_csv(path+file_list[5], skiprows=[0,1,2,3,4,5,6])
thai = pd.read_csv(path+file_list[6], skiprows=[0,1,2,3,4,5,6])
vietnam = pd.read_csv(path+file_list[7], skiprows=[0,1,2,3,4,5,6])
malay = pd.read_csv(path+file_list[8], skiprows=[0,1,2,3,4,5,6])
panama = pd.read_csv(path+file_list[8], skiprows=[0,1,2,3,4,5,6])
peru = pd.read_csv(path+file_list[8], skiprows=[0,1,2,3,4,5,6])
columbia = pd.read_csv(path+file_list[9], skiprows=[0,1,2,3,4,5,6])

In [260]:
china = change(china)
saudi = change(saudi)
india = change(india)
indonesia = change(indonesia)
equador = change(equador)
philipines = change(philipines)
thai = change(thai)
vietnam = change(vietnam)
malay = change(malay)
panama = change(panama)
peru = change(peru)
columbia = change(columbia)

In [261]:
df_shr['CTRY_1'].unique()

array(['사우디아라비아', '에콰도르', '말레이시아', '베트남', '태국', '중국', '인도', '인도네시아',
       '파나마', '페루', '필리핀', '콜롬비아'], dtype=object)

In [262]:
temp = []
for day, crt in zip(df_shr['date'], df_shr['CTRY_1']):
  if crt == "사우디아라비아":
    temp.append(saudi[day])
  elif crt == "에콰도르":
    temp.append(equador[day])
  elif crt == "말레이시아":
    temp.append(malay[day])
  elif crt == "베트남":
    temp.append(vietnam[day])
  elif crt == "태국":
    temp.append(thai[day])
  elif crt == "중국":
    temp.append(china[day])
  elif crt == "인도":
    temp.append(india[day])
  elif crt == "인도네시아":
    temp.append(indonesia[day])
  elif crt == "파나마":
    temp.append(panama[day])
  elif crt == "페루":
    temp.append(peru[day])
  elif crt == "필리핀":
    temp.append(philipines[day])
  else:
    temp.append(columbia[day])

In [263]:
df_shr['temp']=temp

## 흰다리새우 온도 데이터 병합(2020)

In [264]:
path = './shrimp_temp/2020/'
file_list = os.listdir(path)
print ("file_list: {}".format(file_list))

file_list: ['2020thai_temp.csv', '2020china_temp.csv', '2020columbia_temp.csv', '2020vietnam_temp.csv', '2020peru_temp.csv', '2020indonesia_temp.csv', '2020india_temp.csv', '2020malay_temp.csv', '2020equador_temp.csv']


In [265]:
thai = pd.read_csv(path+file_list[0], skiprows=[0,1,2,3,4,5,6])
china = pd.read_csv(path+file_list[1], skiprows=[0,1,2,3,4,5,6])
columbia = pd.read_csv(path+file_list[2], skiprows=[0,1,2,3,4,5,6])
vietnam = pd.read_csv(path+file_list[3], skiprows=[0,1,2,3,4,5,6])
peru = pd.read_csv(path+file_list[4], skiprows=[0,1,2,3,4,5,6])
indonesia = pd.read_csv(path+file_list[5], skiprows=[0,1,2,3,4,5,6])
india = pd.read_csv(path+file_list[6], skiprows=[0,1,2,3,4,5,6])
malay = pd.read_csv(path+file_list[7], skiprows=[0,1,2,3,4,5,6])
equador = pd.read_csv(path+file_list[8], skiprows=[0,1,2,3,4,5,6])

In [266]:
thai = change(thai)
china = change(china)
columbia = change(columbia)
vietnam = change(vietnam)
peru = change(peru)
indonesia = change(indonesia)
india = change(india)
malay = change(malay)
equador = change(equador)

In [267]:
df_shr_val['CTRY_1'].unique()

array(['태국', '베트남', '인도', '말레이시아', '에콰도르', '중국', '페루', '인도네시아', '콜롬비아'],
      dtype=object)

In [268]:
temp = []
for day, crt in zip(df_shr_val['date'], df_shr_val['CTRY_1']):
  if crt == "태국":
    temp.append(thai[day])
  elif crt == "베트남":
    temp.append(vietnam[day])
  elif crt == "인도":
    temp.append(india[day])
  elif crt == "말레이시아":
    temp.append(malay[day])
  elif crt == "에콰도르":
    temp.append(equador[day])
  elif crt == "중국":
    temp.append(china[day])
  elif crt == "페루":
    temp.append(peru[day])
  elif crt == "인도네시아":
    temp.append(indonesia[day])
  else:
    temp.append(columbia[day])

In [269]:
df_shr_val['temp']=temp

# 최종 병합 데이터 저장

In [270]:
path = './merged_data/'
df_sq.to_csv(path+'squid_train.csv', encoding='utf-8')
df_sal.to_csv(path+'salmon_train.csv', encoding='utf-8')
df_shr.to_csv(path+'shrimp_train.csv', encoding='utf-8')
df_sq_val.to_csv(path+'squid_val.csv', encoding='utf-8')
df_sal_val.to_csv(path+'salmon_val.csv', encoding='utf-8')
df_shr_val.to_csv(path+'shrimp_val.csv', encoding='utf-8')